In the cell below enter in the path where your csv files are located and a seperate folder where you want to output the results. Then hit Run All at the top.

If it prompts you to select an enviroment select jupyter-env.

The first time you run this is VS code you may need to go on the extensions tag on the left hand bar and install the jupyter extension

In [ ]:
inputpath = r"YourPathHere"
outputpath = r"YourPathHere"

In [ ]:
import numpy as np
import pandas as pd
import os
import math
import re
from tqdm import tqdm

def distanceBetweenPoints(X1, Y1, X2, Y2):
	"""Calculates the distance between two coordinates

	Args:
		X1 (float): Start X coordinate
		Y1 (float): Start Y coordinate
		X2 (float): End X coordinate
		Y2 (float): End Y coordinate

	Returns:
		float: Distance between the two points
	"""	
	xdiff = X1 - X2
	ydiff = Y1 - Y2
	Distance = math.sqrt((xdiff*xdiff) + (ydiff*ydiff))
	return Distance

def vectorisableDistance(Coord, OtherCoord):
	return distanceBetweenPoints(Coord[0], Coord[1], OtherCoord[0], OtherCoord[1])

def getDistances(dataframe1, dataframe2):
	OutDict = {}
	for i in tqdm(range(len(dataframe1))):
		copieddf = dataframe2.copy()
		X2 = dataframe1["X"][i]
		Y2 = dataframe1["Y"][i]
		copieddf["X2"] = X2
		copieddf["Y2"] = Y2
		copieddf["Distance"] = np.sqrt(((copieddf["X"] - copieddf["X2"])**2) + ((copieddf["Y"] - copieddf["Y2"])**2))
		OutDict[f"{X2}, {Y2}"] = copieddf["Distance"]
	return pd.DataFrame(OutDict)

if not os.path.exists(outputpath):
	os.makedirs(outputpath)

regexpattern = re.compile(r"\.csv$", re.IGNORECASE)
pathlist = [f for f in os.listdir(inputpath) if regexpattern.search(f)]

dfdict = {}
for csvfilename in pathlist:
	f = csvfilename.split("_")
	commonfilename = "_".join(f[:-1])
	if commonfilename not in dfdict:
		dfdict[commonfilename] = {}
	dfdict[commonfilename][f[-1].split(".")[0]] = pd.read_csv(os.path.join(inputpath, csvfilename))

for filename in dfdict:
	RealData = getDistances(dfdict[filename]["1"], dfdict[filename]["2"])
	print(f"Saving {filename}_RealDistances.csv")
	RealData.to_csv(os.path.join(outputpath, f"{filename}_RealDistances.csv"), index=False)
	for set in range(int((dfdict[filename]["random"].shape[1] - 1) / 2)):
		subset = dfdict[filename]["random"].filter(items=[f"X{set+1}", f"Y{set+1}"])
		subset.columns = ["X", "Y"]
		RandomData = getDistances(dfdict[filename]["1"], subset)
		print(f"Saving {filename}_RandomDistances_{set+1}.csv")
		RandomData.to_csv(os.path.join(outputpath, f"{filename}_RandomDistances_{set+1}.csv"), index=False)